<a href="https://colab.research.google.com/github/Jumperinside/Financial/blob/main/FINANCIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, date , timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
def xirr(df, guess=0.05, date_column='date', amount_column='amount'):
    """
    Calculates XIRR from a series of cashflows.
    Needs a dataframe with columns date and amount, customisable through parameters.
    Requires Pandas, NumPy libraries

    Parameters:
    -----------

    df: (pandas dataframe)
    guess: (float) Default = 0.05
      The starting rate at which cash flows will be discounted
    date_column: (str)
      The Name of the column of the dataframe where dates are stored
    amount_column: (str)
      The Name of the column of the dataframe where cash flows are stored



    """

    df = df.sort_values(by=date_column).reset_index(drop=True)

    amounts = df[amount_column].values
    dates = df[date_column].values

    years = np.array(dates-dates[0], dtype='timedelta64[D]').astype(int)/365

    step = 0.05 # Step: percentage rate at which the interest rate will change
    epsilon = 0.0001 # Aproximation to zero, it can be increased or decreased according to how exact do you want the calculation to be
    limit = 1000 # Number of iterations. See method of calculation for XIRR, based on iterations.
    residual = 1 # Starting 'NPV' point for iterations

    # Test for direction of cashflows
    disc_val_1 = np.sum(amounts/((1+guess)**years)) # Discounted value 1: lower rate
    disc_val_2 = np.sum(amounts/((1.05+guess)**years)) # Discounted value 2: higher rate
    mul = 1 if disc_val_2 < disc_val_1 else -1
    # if discounted value 2 < discounted value 1, cashflows are positive (investment). if discounted value 2 > discounted value 1, cash flows are negative (loan)

    #Calculate XIRR
    for i in range(limit): # For each iteration
        prev_residual = residual
        residual = np.sum(amounts/((1+guess)**years)) # sum of discounted cash flows
        if abs(residual) > epsilon: # if NPV > epsilon:
            if np.sign(residual) != np.sign(prev_residual): # if residual did not change sign:
                step /= 2 # divide step by two
            guess = guess + step * np.sign(residual) * mul # starting rate + step * residual * mul (direction of cash flows)
        else:
            return guess

In [ ]:
methods = ['Aleman', 'Frances', 'Americano', 'Bullet']
rate_types = ['TNA' , 'TEA' ]
daycountconv = ['30/360' , '30/365' , 'ACT/360' , 'ACT/365' , 'ACT/ACT']
periodicities = ['Mensual', 'Bimestral', 'Trimestral', 'Cuatrimestral', 'Semestral', 'Anual']

# Method are the different methods you can use (more info on each method in it's specific _init_ block)
# Rate_types are the different type you can convey for the interest rate calculations. TNA is the nominal annual rate, TEA is the equivalent annual rate. It is not usual to negotiate a loan with a referred rate other than these two.
# Daycountconv are the conventions for day count used. Google them for specific calculations
# Periodicities are the periods between two payments the loan is settled upon.

In [ ]:


def ACT_360(n): # Not very used. It is consider in this function but not on the class defined below.
  return n / 360

def ACT_ACT(n , N): # Not very used. It is consider in this function but not on the class defined below.
  return n / N

def ACT_365(n):
  return n / 365

def thirty_360(n):
  return 30 / 360

def thirty_365(n):
  return 30 / 365


def TPeriod(rate , rate_type , n , daycount):
  """
  The function calculates the effective periodic interest rate considering each day count convention used.
  If a period between two payments is of 187 days, this is the rate applied for this 187 days and is derived
  from the Annual Rate Provided, wether it's nominal or effective rate.

    Parameters:
    -----------
    rate: (float)
      The Annual percentage interest rate informed for the instrument
    rate_type: (str)
      The type of percentage interest rate. Only two posibilities:
      * 'TNA'(Annual Nominal Rate)
      * 'TEA' (Annual Effective Rate)
    n: (int)
      The number of days between the period over the interest calculations are to be performed
      and the inmediately prior period.
    daycount: (str)
      The convention applied for day count for the calculations.
      While there are many conventions, most of them are not very frequently used in the financial
      markets. The ones to be used are:
      * ACT/365:
      * 30/365
      * 30/365
      Read more about day count conventions:
      https://en.wikipedia.org/wiki/Day_count_convention


  """


  if daycount not in daycountconv:
    raise ValueError("Not supported day count convention")
  else:
    if rate_type == 'TNA':
      if daycount == 'ACT/365':
        TPeriod = rate * ACT_365(n)
      elif daycount == '30/360':
        TPeriod = rate * thirty_360(n)
      else: # daycount == '30/365'
        TPeriod = rate * thirty_365(n)
    else:
      if daycount == 'ACT/365':
        TPeriod = ((1 + rate) ** (ACT_365(n))) - 1
      elif daycount == '30/360':
        TPeriod = ((1 + rate) ** (thirty_360(n))) - 1
      else: # daycount == '30/365'
        TPeriod = ((1 + rate) ** (thirty_365(n))) - 1
  return TPeriod



In [ ]:
class instrument():
  """
  A class representing a financial instrument, specifically a bank-loan.
  The class provides methods for calculating loan schedules based on different methods, including French, German, American, and Bullet methods.
  While it was developed mainly with the goal of obtaining payments schedules for different kind of loans,
  it could also work with fixed-rent investments (either bought or issued), such as bonds.
  The parameters used for calculations are the those thar are most used in the market. SOME conditions
  could not work or even be defined by the code, such as ACT/360 day count convention or periodic effective interest rate.

  Class instrument

    Attributes:
    -----------
    code: (str)
      A numerical or alfanumerical code that identifies a specific loan.
      It is used when the whole debt is consolidated in a single dataframe
    date: (str)
      The starting day of an instrument, usually conincident with the initial
      credit of the loan.
    method: (str)
      The method used for amortizations. The accepted posibilities are:
      * 'Frances': Method consisting of equal payments with decreasing interest and increasing amortization
      * 'Aleman': Method consisting of equal amortization with decreasing interest and decreasing payments
      * 'Americano': Method consisting of one or more interest-only payments and a single amortization at
        the end of the maturity
      * 'Bullet': Method consisting of a single repayment of 100% amortization and interest
    capital: (float)
      The principal of the loan
    rate: (float)
      The interest rate (not expressed in percentual format)
    rate_type: (str)
      The type of percentage interest rate. Only two posibilities:
      * 'TNA'(Annual Nominal Rate)
      * 'TEA' (Annual Effective Rate)

    Attributes (kwargs):
    -------------------
    These attributes are specific for the method used:
    daycount: (str)
      Day count convention used.
    periodicity: (str)
      The period that separates two payments
    amortizations: (int)
      Number of amortizations for the principal
    coupons (optional): (int)
      Number of interest-only payments
    term: (int)
      Number of days until the final payment

   The code could be imported as a module and requires the import of pandas, datetime and dateutils.relativedelta.




  """
  def __init__(self, code, date, method, capital, rate, rate_type, **kwargs):
    self.code = code # code is the specific number of a single loan. It is usually specified on the loan recipe
    self.date = dt.strptime(str(date), "%d/%m/%Y") # the date the loan is taken
    self.method = method # method of amortization
    self.capital = capital # principal of the loan
    self.rate = rate # interest rate
    if rate_type not in rate_types:
      raise ValueError(f"Select a valid rate type between the following: {rate_types}")
    self.rate_type = rate_type

    if method == 'Frances':
      self._init_french_method(**kwargs)
    elif method == 'Aleman':
      self._init_german_method(**kwargs)
    elif method == 'Americano':
      self._init_american_method(**kwargs)
    elif method == 'Bullet':
      self._init_bullet_method(**kwargs)
    else:
      raise ValueError(f"Unsupported method: {method}")

# **kwargs for each method:

  def _init_french_method(self, daycount, periodicity, amortizations, coupons=0, term=None): # In this method the total payment is constant, while the interests and amortizations vary each payment

    """
    Initialize the French loan calculation method.
    ----------------------------------------------
    Parameters required:
    daycount: (str)
      The day count convention for interest calculations.
      For french method it is required that all periods are equally separated.
      Therefore, accepted parameters for French method:
      * '30/360'
      * '30/365'
    periodicity: (str)
      The periodicity of payments
      * 'Mensual': Monthly separated periods
      * 'Bimestral': Two months separated periods
      * 'Trimestral': Three months separated periods
      * 'Cuatrimestral': Four months separated periods
      * 'Semestral': Six months separated periods
      * 'Anual': Twelve months separated periods
    amortizations: (int)
      The number of amortizations.
    coupons: (int) (Optional)
      The number of interest-only coupons, if they exists
    term: (None)
      Term is not used on frech method.

    """

    if 'daycount' not in locals() or 'periodicity' not in locals():
      raise ValueError("Daycount and periodicity are mandatory for French method.")
    if amortizations is None:
      raise ValueError("Amortizations are required for French method.")
    if daycount == 'ACT/365' or daycount == 'ACT/360' or daycount == 'ACT/ACT':
      raise ValueError("Only 30/360 or 30/365 day count conventions allowed in the French method") # Because ACT/x conventions have uneven periods between installments (e.g. some months have 30 days and some 31), the periodic interest rate varies and therefore a constant payment cannot be calculated
    self.daycount = daycount
    self.periodicity = periodicity
    self.amortizations = amortizations # Number of amortizations
    self.coupons = coupons # Number of interest-only payments
    if term is not None:
      raise ValueError("The term parameter is not allowed in the French method.")
    self.term = None # Term is not used in the French method

    periodicity_to_months = {'Mensual': 1, 'Bimestral': 2, 'Trimestral': 3, 'Cuatrimestral': 4, 'Semestral': 6 , 'Anual':12}
    self.months = periodicity_to_months.get(self.periodicity) # How many months transcur between two payments for each periodicity


  def _init_german_method(self, daycount, periodicity, amortizations, coupons=0, term=None): # In this method, the amortizations are all constant while interests and total payments vary each installment

    """
    Initialize the German loan calculation method.
    ----------------------------------------------
    Parameters required:
    daycount: (str)
      The day count convention for interest calculations.
      * '30/360'
      * '30/365'
      * 'ACT/365'
    periodicity: (str)
      The periodicity of payments
      * 'Mensual': Monthly separated periods
      * 'Bimestral': Two months separated periods
      * 'Trimestral': Three months separated periods
      * 'Cuatrimestral': Four months separated periods
      * 'Semestral': Six months separated periods
      * 'Anual': Twelve months separated periods
    amortizations: (int)
      The number of amortizations.
    coupons: (int) (Optional)
      The number of interest-only coupons, if they exists
    term: (None)
      Term is not used on german method.

    """

    if 'daycount' not in locals() or 'periodicity' not in locals():
      raise ValueError("Daycount and periodicity are mandatory for German method.")
    if amortizations is None:
      raise ValueError("Amortizations are required for German method.")
    self.daycount = daycount
    self.periodicity = periodicity
    self.amortizations = amortizations
    self.coupons = coupons
    if term is not None:
      raise ValueError("The term parameter is not allowed in the German method.")
    self.term = None # Term is not used in the German method

    periodicity_to_months = {'Mensual': 1, 'Bimestral': 2, 'Trimestral': 3, 'Cuatrimestral': 4, 'Semestral': 6 , 'Anual':12}
    self.months = periodicity_to_months.get(self.periodicity)


  def _init_american_method(self, daycount, periodicity, coupons, term=None , amortizations=None): # In this method we have one or more interest-only installment and one single amortization at the end of the loan maturity

    """
    Initialize the American loan calculation method.
    ------------------------------------------------
    Parameters required:
    daycount: (str)
      The day count convention for interest calculations.
      * '30/360'
      * '30/365'
      * 'ACT/365'
    periodicity: (str)
      The periodicity of payments
      * 'Mensual': Monthly separated periods
      * 'Bimestral': Two months separated periods
      * 'Trimestral': Three months separated periods
      * 'Cuatrimestral': Four months separated periods
      * 'Semestral': Six months separated periods
      * 'Anual': Twelve months separated periods
    amortizations: (None)
      Number of amortizations is not used on American method.
    coupons: (int)
      The number of interest-only coupons, must be at least 1.
    term: (None)
      Term is not used on american method.

    """

    if 'coupons' not in locals():
      raise ValueError("Coupons are mandatory for American method.")
    self.daycount = daycount
    self.periodicity = periodicity
    if amortizations is not None:
      raise ValueError("The amortizations parameter is not allowed in the American method")
    self.amortizations = None # No amortizations in the American method
    self.coupons = coupons
    if term is not None:
      raise ValueError("The term parameter is not allowed in the American method.")
    self.term = None # Term is not used in the American method

    periodicity_to_months = {'Mensual': 1, 'Bimestral': 2, 'Trimestral': 3, 'Cuatrimestral': 4, 'Semestral': 6 , 'Anual':12}
    self.months = periodicity_to_months.get(self.periodicity)


  def _init_bullet_method(self, term): # For this simple method, we have only one final interest + amortization installment and no interest-only payments. It is often negotiated at a simple term of n days from the initial date.

    """
    Initialize the Bullet loan calculation method.
    ------------------------------------------------
    Parameters required:
    daycount: (None)
      The daycount parameter is not used on Bullet method
    periodicity: (None)
      The periodicity parameter is not used on Bullet method.
    amortizations: (None)
      Number of amortizations is not used on Bullet method.
    coupons: (None)
      The number of coupons is not used on Bullet method.
    term: (int)
      The number of days to maturity of the instrument.

    """


    if 'term' not in locals():
      raise ValueError("Term is mandatory for Bullet method.")
    self.daycount = None  # Daycount is not used in the Bullet method
    self.periodicity = None  # Periodicity is not used in the Bullet method
    self.amortizations = None  # No amortizations in the Bullet method
    self.coupons = None  # No coupons in the Bullet method
    self.term = term


  def french_method(self):

    """
    Method to obtain a amortization schedule based on the French amortization method.

    """
    rows = [{'payment': 0, 'loan_number': self.code, 'date': self.date, 'outstanding': self.capital , 'amortization': self.capital, 'interest': 0, 'total_payment': self.capital}]
    if self.coupons and self.coupons > 0:
      for i in range(self.coupons):
        loan_number = self.code
        if self.daycount == 'ACT/365' or self.daycount == 'ACT/360' or self.daycount == 'ACT/ACT':
          raise ValueError("Only 30/360 or 30/365 day count conventions allowed in the French method")
        else:
          date = rows[-1]['date'] + relativedelta(days = 30 * self.months)
        n = (date - rows[-1]['date']).days
        if self.rate_type == 'TNA':
          effectiverate = TPeriod(self.rate , self.rate_type , n , self.daycount) * self.months
        else:
          if self.daycount == '30/360':
            effectiverate =  ((1 + self.rate) ** (self.months * 30 / 360)) - 1
          else:
            effectiverate = ((1 + self.rate) ** (self.months * 30 / 365)) - 1
        amortization = 0
        outstanding = round(amortization + rows[-1]['outstanding'] , 2)
        interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
        total_payment = interest
        row = {'payment': i + 1 , 'loan_number': loan_number , 'date': date, 'outstanding': outstanding, 'amortization': amortization , 'interest': interest ,  'total_payment': total_payment }
        rows.append(row)
    else:
      pass

    for i in range(self.amortizations):
      loan_number = self.code
      if self.daycount == 'ACT/ACT' or self.daycount == 'ACT/365':
        raise ValueError("Only 30/360 or 30/365 day count conventions allowed in the French method")
      else:
        date = rows[-1]['date'] + relativedelta(days = 30 * self.months)
      n = (date - rows[-1]['date']).days
      if self.rate_type == 'TNA':
        effectiverate = TPeriod(self.rate , self.rate_type , n , self.daycount) * self.months
      else:
        if self.daycount == '30/360':
          effectiverate =  ((1 + self.rate) ** (self.months * 30 / 360)) - 1
        else:
          effectiverate = ((1 + self.rate) ** (self.months * 30 / 365)) - 1
      total_payment = round(-self.capital * ((effectiverate * (1+ effectiverate) ** self.amortizations) / ((1 + effectiverate) ** self.amortizations - 1)) , 2)
      interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
      amortization = round(total_payment - interest , 2)
      outstanding = round(amortization + rows[-1]['outstanding'] , 2)
      row = {'payment': (i + self.coupons + 1) , 'loan_number': loan_number , 'date': date, 'outstanding': outstanding, 'amortization': amortization , 'interest': interest ,  'total_payment': total_payment }
      rows.append(row)
    instrument_df = pd.DataFrame(rows)
    return instrument_df

  def german_method(self):

    """
    Method to obtain an amortization schedule based on the German amortization method.

    """
    rows = [{'payment': 0, 'loan_number': self.code, 'date': self.date, 'outstanding': self.capital , 'amortization': self.capital, 'interest': 0, 'total_payment': self.capital}]
    if self.coupons and self.coupons > 0:
      for i in range(self.coupons):
        loan_number = self.code
        if self.daycount == 'ACT/ACT' or self.daycount == 'ACT/365':
          date = self.date + relativedelta(months = (i + 1) * self.months)
        else:
          date = rows[-1]['date'] + relativedelta(days = 30 * self.months)
        n = (date - rows[-1]['date']).days
        effectiverate = TPeriod(self.rate , self.rate_type , n , 'ACT/365') # Day count is always ACT/365 for German method for interests calculation purpose
        amortization = 0
        outstanding = round(amortization + rows[-1]['outstanding'] , 2)
        interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
        total_payment = interest
        row = {'payment': i + 1, 'loan_number': loan_number , 'date': date, 'outstanding': outstanding, 'amortization': amortization , 'interest': interest ,  'total_payment': total_payment }
        rows.append(row)
    else:
      pass

    for i in range(self.amortizations):
      loan_number = self.code
      if self.daycount == 'ACT/ACT' or self.daycount == 'ACT/365':
        date = self.date + relativedelta(months = (i + 1 + self.coupons) * self.months)
      else:
        date = rows[-1]['date'] + relativedelta(days = 30 * self.months)
      n = (date - rows[-1]['date']).days
      effectiverate = TPeriod(self.rate , self.rate_type , n , 'ACT/365') # Day count is always ACT/365 for German method for interests calculation purpose
      amortization = round(-self.capital / self.amortizations , 2)
      outstanding = round(amortization + rows[-1]['outstanding'] , 2)
      interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
      total_payment = round(amortization + interest , 2)
      row = {'payment': (i + self.coupons) , 'loan_number': loan_number , 'date': date, 'outstanding': outstanding, 'amortization': amortization , 'interest': interest ,  'total_payment': total_payment }
      rows.append(row)
    instrument_df = pd.DataFrame(rows)
    return instrument_df

  def american_method(self):

    """
    Method to obtain an amortization schedule based on the American amortization method.

    """
    rows = [{'payment': 0, 'loan_number': self.code, 'date': self.date, 'outstanding': self.capital , 'amortization': self.capital, 'interest': 0, 'total_payment': self.capital}]
    if not self.coupons or self.coupons < 1 or self.amortizations:
      raise ValueError(f"The american method consists of a single amortization, (you entered {self.amortizations}) with at least 1 interest coupon payment (you entered {self.coupons})")
    else:
      for i in range(self.coupons):
        loan_number = self.code
        if self.daycount == 'ACT/ACT' or self.daycount == 'ACT/365':
          date = self.date + relativedelta(months = (i + 1) * self.months)
        else:
          date = rows[-1]['date'] + relativedelta(days = 30 * self.months)
        n = (date - rows[-1]['date']).days
        effectiverate = TPeriod(self.rate , self.rate_type , n , 'ACT/365') # Day count is always ACT/365 for American method
        amortization = 0
        outstanding = round(amortization + rows[-1]['outstanding'] , 2)
        interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
        total_payment = interest
        row = {'payment': i + 1, 'loan_number': loan_number , 'date': date, 'outstanding': outstanding, 'amortization': amortization , 'interest': interest ,  'total_payment': total_payment }
        rows.append(row)

      loan_number = self.code
      if self.daycount == 'ACT/ACT' or self.daycount == 'ACT/365':
        date = rows[-1]['date'] + relativedelta(months = self.months)
      else:
        date = rows[-1]['date'] + relativedelta(days = 30 * self.months)
      n = (date - rows[-1]['date']).days
      amortization = round(-self.capital , 2)
      effectiverate = TPeriod(self.rate , self.rate_type , n , 'ACT/365') # Day count is always ACT/365 for American method
      interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
      total_payment = round(amortization + interest , 2)
      outstanding = round(amortization + rows[-1]['outstanding'], 2)
      row = {'payment': self.coupons + 1, 'loan_number': loan_number, 'date': date, 'outstanding': outstanding, 'amortization': amortization, 'interest': interest, 'total_payment': total_payment}
      rows.append(row)
    instrument_df = pd.DataFrame(rows)
    return instrument_df

  def bullet_method(self):

    """
    Method to obtain an amortization schedule based on the Bullet amortization method.

    """
    rows = [{'payment': 0, 'loan_number': self.code, 'date': self.date, 'outstanding': self.capital , 'amortization': self.capital, 'interest': 0, 'total_payment': self.capital}]
    if (not self.term or self.term == 0) or self.coupons or self.amortizations:
      raise ValueError(f"The Bullet method consists of a single amortization, you entered {self.amortizations}. It cannot have coupons and you must provide a term in days, you entered {self.term}")
    else:
      loan_number = self.code
      date = self.date + relativedelta(days = self.term)
      amortization =  round(-self.capital , 2)
      outstanding = round(amortization + rows[-1]['outstanding'] , 2)
      effectiverate = TPeriod(self.rate , self.rate_type , self.term , 'ACT/365') # Day count is always ACT/365 for Bullet method
      interest = round(-(rows[-1]['outstanding'] * effectiverate) , 2)
      total_payment = round(amortization + interest , 2)
      row = {'payment': 1 , 'loan_number': loan_number , 'date': date, 'outstanding': outstanding, 'amortization': amortization , 'interest': interest ,  'total_payment': total_payment }
      rows.append(row)
    instrument_df = pd.DataFrame(rows)
    return instrument_df


  def schedule(self):

    """
    Method to obtain a installments schedule based on the amortization system passed as a parameter.
    Consists on the following columns:
    payment:
      The number of each payment, starting on 0 that is the initial loan credit by the loaner
    loan_number:
      A numerical or alfanumerical code that identifies a specific loan.
    date:
      date due for each payment
    outstanding:
      the outstanding capital remaining at the end of each period after the payment is done
    amortization:
      The amortization paid on each installment
    interest:
      The interest paid on each installment
    total_payment:
      The sum of amortization and interest paid on each installment

    """

    if self.method == 'Frances':
      return self.french_method()
    elif self.method == 'Aleman':
      return self.german_method()
    elif self.method == 'Americano':
      return self.american_method()
    else:
      return self.bullet_method()


In [ ]:
# MÉTODOS PROBADOS EXITOSAMENTE (COMPARAR SALIDA CONTRA EXCEL DE RESPALDO) - PROBAR CON MÉTODO: nombre.schedule()

frances1 = instrument('frances1' , '10/12/2022' , 'Frances' , 1000000 , 0.6 , 'TNA' , periodicity = 'Mensual' , amortizations = 12 , daycount = '30/360')
frances2 = instrument('frances2' , '11/03/2021' , 'Frances' , 241700000 , 0.3 , 'TNA' , periodicity = 'Mensual' , daycount = '30/365' , amortizations = 45 , coupons = 3)
frances3 = instrument('frances3' , '19/06/2023' , 'Frances' , 2500000 , 0.65 , 'TEA' , periodicity = 'Bimestral' , daycount = '30/365' , amortizations = 12)
aleman1 = instrument('aleman1' , '31/08/2022' , 'Aleman' , 114781326 , 0.609723757042777 , 'TNA' , periodicity = 'Semestral' ,  amortizations = 8 , daycount = 'ACT/365')
aleman2 = instrument('aleman2' , '22/02/2022' , 'Aleman' , 5650000 , 0.5 , 'TEA' ,  periodicity = 'Mensual' , amortizations = 12 , coupons = 2 , daycount = '30/360')
aleman3 = instrument('aleman3' , '10/12/2023' , 'Aleman' , 4350000 , 0.36 , 'TEA' , periodicity = 'Bimestral' , coupons = 3 , amortizations = 10 , daycount = '30/365')
americano1 = instrument('americano1' , '25/09/2023' , 'Americano' , 148000000 , 1.3 , 'TNA' , periodicity = 'Trimestral' , coupons = 2 , daycount = 'ACT/365')
americano2 = instrument('americano2' , '29/09/2023' , 'Americano' , 190000000 , 1.3 , 'TNA' , periodicity = 'Semestral' , coupons = 1 , daycount = 'ACT/365')
americano3 = instrument('americano3' , '10/11/2023' , 'Americano' , 1000000 , 0.8 , 'TEA' , periodicity = 'Mensual' , coupons = 2 , daycount = '30/360')
bullet1 = instrument('bullet1' , '09/12/2018' , 'Bullet' , 1000000 , 0.6 , 'TNA' , term = 180)

In [ ]:
# EJEMPLO DE MODIFICACIÓN DE VALORES OBTENIDOS:

bullet2 = instrument('bullet1' , '09/12/2018' , 'Bullet' , 1000000 , 0.6 , 'TNA' , term = 180)
bullet2.schedule()

bullet2mod = bullet2.schedule()
bullet2mod['date'][1] = dt.strptime(str('09/06/2019'), "%d/%m/%Y")
bullet2mod['interest'][1] = -bullet2mod['outstanding'][0] * TPeriod(bullet2.rate , bullet2.rate_type , ((bullet2mod['date'][1] - bullet2mod['date'][0]).days) , 'ACT/365')
bullet2mod['total_payment'][1] = round(bullet2mod['amortization'][1] + bullet2mod['interest'][1] , 2)
bullet2mod

<ipython-input-7-b66c5a3f937a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bullet2mod['date'][1] = dt.strptime(str('09/06/2019'), "%d/%m/%Y")
<ipython-input-7-b66c5a3f937a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bullet2mod['interest'][1] = -bullet2mod['outstanding'][0] * TPeriod(bullet2.rate , bullet2.rate_type , ((bullet2mod['date'][1] - bullet2mod['date'][0]).days) , 'ACT/365')
<ipython-input-7-b66c5a3f937a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

,payment,loan_number,date,outstanding,amortization,interest,total_payment
0,0,bullet1,2018-12-09,1000000,1000000,0.000000,1000000.00
1,1,bullet1,2019-06-09,0,-1000000,-299178.082192,-1299178.08


In [ ]:
# EJEMPLO DE ALMACENAMIENTO DE PRÉSTAMOS EN UNA SÓLA DF:

frances1 = instrument('frances1' , '10/12/2022' , 'Frances' , 1000000 , 0.6 , 'TNA' , periodicity = 'Mensual' , amortizations = 12 , daycount = '30/360')
deuda = pd.concat([bullet2mod , frances1.schedule()])
deuda

,payment,loan_number,date,outstanding,amortization,interest,total_payment
0,0,bullet1,2018-12-09,1000000.00,1000000.00,0.000000,1000000.00
1,1,bullet1,2019-06-09,0.00,-1000000.00,-299178.082192,-1299178.08
0,0,frances1,2022-12-10,1000000.00,1000000.00,0.000000,1000000.00
1,1,frances1,2023-01-09,937174.59,-62825.41,-50000.000000,-112825.41
2,2,frances1,2023-02-08,871207.91,-65966.68,-46858.730000,-112825.41
3,3,frances1,2023-03-10,801942.90,-69265.01,-43560.400000,-112825.41
4,4,frances1,2023-04-09,729214.63,-72728.27,-40097.140000,-112825.41
5,5,frances1,2023-05-09,652849.95,-76364.68,-36460.730000,-112825.41
6,6,frances1,2023-06-08,572667.04,-80182.91,-32642.500000,-112825.41
7,7,frances1,2023-07-08,488474.98,-84192.06,-28633.350000,-112825.41


In [ ]:
frances1 = instrument('frances1' , '10/12/2022' , 'Frances' , 1000000 , 0.6 , 'TNA' , periodicity = 'Mensual' , amortizations = 12 , daycount = '30/360')
frances1.schedule()

,payment,loan_number,date,outstanding,amortization,interest,total_payment
0,0,frances1,2022-12-10,1000000.00,1000000.00,0.00,1000000.00
1,1,frances1,2023-01-09,937174.59,-62825.41,-50000.00,-112825.41
2,2,frances1,2023-02-08,871207.91,-65966.68,-46858.73,-112825.41
3,3,frances1,2023-03-10,801942.90,-69265.01,-43560.40,-112825.41
4,4,frances1,2023-04-09,729214.63,-72728.27,-40097.14,-112825.41
5,5,frances1,2023-05-09,652849.95,-76364.68,-36460.73,-112825.41
6,6,frances1,2023-06-08,572667.04,-80182.91,-32642.50,-112825.41
7,7,frances1,2023-07-08,488474.98,-84192.06,-28633.35,-112825.41
8,8,frances1,2023-08-07,400073.32,-88401.66,-24423.75,-112825.41
9,9,frances1,2023-09-06,307251.58,-92821.74,-20003.67,-112825.41


In [ ]:
frances2 = instrument('frances2' , '11/03/2021' , 'Frances' , 241700000 , 0.3 , 'TNA' , periodicity = 'Mensual' , daycount = '30/365' , amortizations = 45 , coupons = 3)
frances2.schedule()

,payment,loan_number,date,outstanding,amortization,interest,total_payment
0,0,frances2,2021-03-11,2.417000e+08,2.417000e+08,0.00,2.417000e+08
1,1,frances2,2021-04-10,2.417000e+08,0.000000e+00,-5959726.03,-5.959726e+06
2,2,frances2,2021-05-10,2.417000e+08,0.000000e+00,-5959726.03,-5.959726e+06
3,3,frances2,2021-06-09,2.417000e+08,0.000000e+00,-5959726.03,-5.959726e+06
4,4,frances2,2021-07-09,2.387090e+08,-2.990986e+06,-5959726.03,-8.950712e+06
5,5,frances2,2021-08-08,2.356443e+08,-3.064736e+06,-5885975.69,-8.950712e+06
6,6,frances2,2021-09-07,2.325040e+08,-3.140305e+06,-5810406.84,-8.950712e+06
7,7,frances2,2021-10-07,2.292862e+08,-3.217737e+06,-5732974.66,-8.950712e+06
8,8,frances2,2021-11-06,2.259892e+08,-3.297079e+06,-5653633.19,-8.950712e+06
9,9,frances2,2021-12-06,2.226108e+08,-3.378377e+06,-5572335.35,-8.950712e+06


In [ ]:
aleman1 = instrument('aleman1' , '31/08/2022' , 'Aleman' , 114781326 , 0.609723757042777 , 'TNA' , periodicity = 'Semestral' ,  amortizations = 8 , daycount = 'ACT/365')
aleman1.schedule()

,payment,loan_number,date,outstanding,amortization,interest,total_payment
0,0,aleman1,2022-08-31,1.147813e+08,1.147813e+08,0.00,1.147813e+08
1,0,aleman1,2023-02-28,1.004337e+08,-1.434767e+07,-34704841.48,-4.905251e+07
2,1,aleman1,2023-08-31,8.608599e+07,-1.434767e+07,-30870052.37,-4.521772e+07
3,2,aleman1,2024-02-29,7.173833e+07,-1.434767e+07,-26172435.70,-4.052010e+07
4,3,aleman1,2024-08-31,5.739066e+07,-1.434767e+07,-22050037.40,-3.639770e+07
5,4,aleman1,2025-02-28,4.304300e+07,-1.434767e+07,-17352420.74,-3.170009e+07
6,5,aleman1,2025-08-31,2.869533e+07,-1.434767e+07,-13230022.44,-2.757769e+07
7,6,aleman1,2026-02-28,1.434767e+07,-1.434767e+07,-8676210.37,-2.302388e+07
8,7,aleman1,2026-08-31,0.000000e+00,-1.434767e+07,-4410007.48,-1.875767e+07
